In [1]:
%cd /mnt/storage/maxenis/coptic-machine-translation

/mnt/storage/maxenis/coptic-machine-translation


In [2]:
import pandas as pd
import datasets

In [215]:
dataset = datasets.load_dataset("bbaw_egyptian")

In [216]:
data = dataset['train'].to_pandas()

In [217]:
data.head()

,transcription,translation,hieroglyphs
0,"⸢pḏ,wt-9⸣ n =f [⸮ḥtr?] ⸢m⸣",... die Neun-Bogenvölker ... zu ihm ... Pferde...,
1,"ḥtr tp,j ꜥꜣ n ḥm =f Ꜥꜣ-nḫt,w",[Pferde]: Das erste große Pfergespann seiner M...,
2,"⸢wr⸣.pl ⸢ꜥꜣi̯⸣.pl n.w ⸢Rṯn,w⸣ ⸢jni̯⸣ ⸢ḥm⸣ ⸢=f⸣...",[Gefangene]: Die großen Oberhäupter von Retjen...,
3,"ḏd-mdw jn ꜣs,t wr,t mw,t-nṯr","Worte sprechen durch Isis, die Große, die Gott...",
4,"sꜣ =j mri̯ =j Rꜥw-msi̯-sw-mri̯-Jmn ⸢ꜥḥꜥ,w⸣ =k ...","""Mein geliebter Sohn Ramses-meriamun, deine Le...",


In [218]:
data = data[~data["transcription"].isnull()]

In [219]:
data = data[~data["transcription"].str.contains(":|≡|~|\(|\)|〈|〈|\[|\]|\{|\}|⸢|⸣|⸮|\?|=")]

In [220]:
data.head()

,transcription,translation,hieroglyphs
3,"ḏd-mdw jn ꜣs,t wr,t mw,t-nṯr","Worte sprechen durch Isis, die Große, die Gott...",
10,"jtn pri̯ ṯꜣ,pl ḥꜣ,tj n jm,j Jwn,w","Die Sonnenscheibe, die herauskommt, der Windha...",
28,"Mn-Mꜣꜥ,t-Rꜥw Stẖ,y-mr-n-Jmn","[König] Men-Maat-Re, Sethos-mer-en-Imen",
29,"Nḫb,t ḥḏ,t-Nḫn","[Geiergöttin] Nechbet, die Weiße von Nechen",
32,"Ḥr,w Kꜣ-nḫt-ḫꜥ-m-Wꜣs,t-sꜥnḫ-tꜣ,wj nswt-bj,tj ...",[König und rhetorischer Text]: Horus: Ka-nacht...,


In [221]:
ROMANIZE_MAP = {
    "ꜣ": "a",
    "ꜥ": "a",
    "ḥ": "h",
    "ẖ": "kh",
    "ḫ": "kh",
    "ẖ": "kh",
    "š": "sh",
    "ṯ": "ch",
    "ḏ": "dg",
    "i̯": "i",
    "ʾ": "a",
    "u̯": "u",
    "ṱ": "t",
}

In [222]:
DEROMANIZE_MAP = {v: k for k, v in ROMANIZE_MAP.items()}

In [223]:
data = data.rename(columns={"transcription": "norm_group"})

In [224]:
# Replace "-" with " "
data["norm"] = data["norm_group"].str.replace("-", " ")

In [225]:
# Replace "-" with "" in norm_group
data["norm_group"] = data["norm_group"].str.replace("-", "")

In [226]:
# make all data in norm and norm_group lower case
data["norm"] = data["norm"].str.lower()
data["norm_group"] = data["norm_group"].str.lower()

In [227]:
# map letters to their romanized counterparts in both norm_group and norm
for k, v in ROMANIZE_MAP.items():
    data["norm_group"] = data["norm_group"].str.replace(k, v)
    data["norm"] = data["norm"].str.replace(k, v)

In [228]:
data["norm"] = data["norm"].str.replace(",", ", ")
data["norm"] = data["norm"].str.replace(r"\,\s+", ", ", regex=True)
# do same with norm_group
data["norm_group"] = data["norm_group"].str.replace(",", ", ")
data["norm_group"] = data["norm_group"].str.replace(r"\,\s+", ", ", regex=True)

# Do the same with periods
data["norm"] = data["norm"].str.replace(".", ". ")
data["norm"] = data["norm"].str.replace(r"\.\s+", ". ", regex=True)
# do same with norm_group
data["norm_group"] = data["norm_group"].str.replace(".", ". ")
data["norm_group"] = data["norm_group"].str.replace(r"\.\s+", ". ", regex=True)

In [229]:
# strip whitespace
data["norm"] = data["norm"].str.strip()
data["norm_group"] = data["norm_group"].str.strip()

In [230]:
data.head()

,norm_group,translation,hieroglyphs,norm
3,"dgdmdw jn as, t wr, t mw, tnchr","Worte sprechen durch Isis, die Große, die Gott...",,"dgd mdw jn as, t wr, t mw, t nchr"
10,"jtn pri cha, pl ha, tj n jm, j jwn, w","Die Sonnenscheibe, die herauskommt, der Windha...",,"jtn pri cha, pl ha, tj n jm, j jwn, w"
28,"mnmaa, traw stkh, ymrnjmn","[König] Men-Maat-Re, Sethos-mer-en-Imen",,"mn maa, t raw stkh, y mr n jmn"
29,"nkhb, t hdg, tnkhn","[Geiergöttin] Nechbet, die Weiße von Nechen",,"nkhb, t hdg, t nkhn"
32,"hr, w kankhtkhamwas, tsankhta, wj nswtbj, tj ...",[König und rhetorischer Text]: Horus: Ka-nacht...,,"hr, w ka nkht kha m was, t sankh ta, wj nswt ..."


In [231]:
from parse_data import illegal

In [232]:
legal = illegal.copy()
legal.add(",")
legal.add(" ")
legal.add(".")
legal.add("!")
for i in range(100):
    legal.add(str(i))

In [234]:
data = data[~data["norm_group"].apply(lambda x: legal.union(set(x)) != legal)]


In [237]:
data.head()

,norm_group,translation,hieroglyphs,norm
3,"dgdmdw jn as, t wr, t mw, tnchr","Worte sprechen durch Isis, die Große, die Gott...",,"dgd mdw jn as, t wr, t mw, t nchr"
10,"jtn pri cha, pl ha, tj n jm, j jwn, w","Die Sonnenscheibe, die herauskommt, der Windha...",,"jtn pri cha, pl ha, tj n jm, j jwn, w"
28,"mnmaa, traw stkh, ymrnjmn","[König] Men-Maat-Re, Sethos-mer-en-Imen",,"mn maa, t raw stkh, y mr n jmn"
29,"nkhb, t hdg, tnkhn","[Geiergöttin] Nechbet, die Weiße von Nechen",,"nkhb, t hdg, t nkhn"
32,"hr, w kankhtkhamwas, tsankhta, wj nswtbj, tj ...",[König und rhetorischer Text]: Horus: Ka-nacht...,,"hr, w ka nkht kha m was, t sankh ta, wj nswt ..."


In [238]:
data.to_csv("datasets/heiroglyph_data.csv", index=False)

In [254]:
data = pd.read_csv("datasets/heiroglyph_data.csv")

In [255]:
data = data[~data["translation"].isnull()]

In [256]:
data = data[~data["translation"].str.contains("\.\.\.")]

In [258]:
data.to_csv("datasets/clean_heiroglyph_data.csv")

In [267]:
# remove duplicates with just one entry
data = data.drop_duplicates(subset=["norm_group"])


In [268]:
data.to_csv("datasets/clean_heiroglyph_data.csv", index=False)